In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json

In [2]:
playtest5_data = pd.read_csv("2018-11-14_Playtest/anonymized_playtest5_data.csv", sep=";")
playtest6_data = pd.read_csv("2019-01-07_Playtest/anonymized_playtest6_data.csv", sep=";")
playtest7_data = pd.read_csv("2019-01-31_Playtest/anonymized_playtest7_data.csv", sep=";")

In [3]:
all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])

In [4]:
all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])

In [5]:
all_playtest_data = all_playtest_data.sort_values('time')

In [6]:
data_by_user = all_playtest_data.groupby('session_id').agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'n_events',
                                                                                              'type':'n_different_events'})

In [7]:
data_by_user['active_time'] = np.nan
data_by_user['avg_dt'] = np.nan
data_by_user['std_dt'] = np.nan

In [8]:
data_by_user.index = data_by_user['session_id'].values

In [9]:
for user in data_by_user['session_id'].unique():
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
   
    # Computing active time
    previousEvent = None
    theresHoldActivity = 15 # np.percentile(allDifferences, 98) is 10 seconds
    activeTime = []
    
    for enum, event in user_events.iterrows():
        
        # If it is the first event
        if(previousEvent is None):
            previousEvent = event
            continue
        
        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
        if(~(delta_seconds > theresHoldActivity)):
            activeTime.append(delta_seconds)
        
        previousEvent = event
        
    data_by_user.at[user, 'active_time'] = round(np.sum(activeTime)/60,2)
    data_by_user.at[user, 'avg_dt'] = round(np.mean(activeTime)/60,2)
    data_by_user.at[user, 'std_dt'] = round(np.std(activeTime)/60,2)

# Difficulty measure

In [33]:
playtest7_config = json.loads(open('2019-01-31_Playtest/StreamingAssets/config.json').read())

In [54]:
json.loads(user_events.loc[2240]['data'])['task_id']

'One Box'

In [72]:
listMovementEvents = ['ws-move_shape', 'ws-rotate_shape', 'ws-scale_shape', 
                      'ws-check_solution', 'ws-undo_action', 'ws-redo_action',
                      'ws-rotate_view', 'ws-snapshot']

In [108]:
# puzzleDict[user~puzzle_id] = {}
puzzleDict = {}

for user in playtest5_data['session_id'].unique():

    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
    # Analyze when a puzzle has been started
    activePuzzle = None
    previousEvent = None
    numberActions = 0
    numberAttempts = 0
    activeTime = []

    for enum, event in user_events.iterrows():
        print(('{} - {}').format(event['time'], event['type']))
        if(event['type'] == 'ws-start_level'):
            print('\\start level\\')
            print(json.loads(event['data']))
            activePuzzle = json.loads(event['data'])['task_id']

        # If event is puzzle complete we always add it
        if(event['type'] == 'ws-puzzle_complete'):
            puzzleDict[event['session_id'] + '~' + activePuzzle]['n_complete'] += 1

        # If they are not playing a puzzle we do not do anything and continue
        if(activePuzzle is None):
            continue

        key = event['session_id'] + '~' + activePuzzle
        if(key not in puzzleDict.keys()):
            puzzleDict[key] = dict()
            puzzleDict[key]['active_time'] = 0
            puzzleDict[key]['n_actions'] = 0
            puzzleDict[key]['n_attempts'] = 0
            puzzleDict[key]['n_complete'] = 0

        # If it is the first event we store the current event and continue
        if(previousEvent is None):
            previousEvent = event
            continue

        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
        if(~(delta_seconds > theresHoldActivity)):
            activeTime.append(delta_seconds)

        if(event['type'] in listMovementEvents):
            numberActions += 1

        if(event['type'] == 'ws-check_solution'):
            numberAttempts += 1

    # Analyze when puzzle is finished or user left
        # Measure time, attempts, completion and actions
        if(event['type'] in ['ws-puzzle_complete', 'ws-exit_to_menu', 'ws-disconnect']):
            print('\\finish\\')
            # time spent
            print('{} minutes, {} actions, {} attempts'.format(round(np.sum(activeTime)/60,2), numberActions, numberAttempts))
            # adding counters
            puzzleDict[key]['active_time'] += round(np.sum(activeTime)/60,2)
            puzzleDict[key]['n_actions'] += numberActions
            puzzleDict[key]['n_attempts'] += numberAttempts

            # reset counters
            previousEvent = None
            activeTime = []
            activePuzzle = None
            numberActions = 0
            numberAttempts = 0

        previousEvent = event
        

2018-11-14 17:01:31.569079 - ws-create_user
2018-11-14 17:01:48.396592 - ws-puzzle_started
2018-11-14 17:01:49.929082 - ws-start_level
\start level\
{'task_id': 'Welcome!', 'set_id': 'Tutorial', 'fullscreen': True, 'resolution': {'x': 1366, 'y': 768}, 'conditions': '{"shapeLimits":[-1,1,0,0,0,0,0],"allowScale":true,"allowRotate":true,"gridDim":3}'}
2018-11-14 17:01:50.075588 - ws-click_nothing
2018-11-14 17:02:18.942371 - ws-click_nothing
2018-11-14 17:02:20.977050 - ws-mode_change
2018-11-14 17:02:22.954830 - ws-create_shape
2018-11-14 17:02:43.676962 - ws-move_shape
2018-11-14 17:02:50.131146 - ws-mode_change
2018-11-14 17:02:56.356076 - ws-check_solution
2018-11-14 17:02:56.505879 - ws-puzzle_complete
\finish\
1.11 minutes, 2 actions, 1 attempts
2018-11-14 17:03:00.092727 - ws-exit_to_menu
2018-11-14 17:03:03.872033 - ws-puzzle_started
2018-11-14 17:03:04.286835 - ws-start_level
\start level\
{'task_id': 'Moving Objects', 'set_id': 'Tutorial', 'fullscreen': True, 'resolution': {'x':

2018-11-14 17:26:41.857929 - ws-restart_puzzle
2018-11-14 17:26:42.012084 - ws-puzzle_started
2018-11-14 17:27:41.311766 - ws-mode_change
2018-11-14 17:27:43.651703 - ws-create_shape
2018-11-14 17:27:44.391990 - ws-create_shape
2018-11-14 17:27:45.281047 - ws-create_shape
2018-11-14 17:27:46.191282 - ws-create_shape
2018-11-14 17:27:49.239150 - ws-mode_change
2018-11-14 17:27:49.967300 - ws-delete_shape
2018-11-14 17:27:50.819662 - ws-select_shape
2018-11-14 17:27:52.550229 - ws-mode_change
2018-11-14 17:27:54.857733 - ws-delete_shape
2018-11-14 17:27:56.411859 - ws-mode_change
2018-11-14 17:27:57.807695 - ws-delete_shape
2018-11-14 17:27:58.945459 - ws-select_shape
2018-11-14 17:28:00.333302 - ws-mode_change
2018-11-14 17:28:01.059926 - ws-delete_shape
2018-11-14 17:30:15.638213 - ws-mode_change
2018-11-14 17:30:17.065967 - ws-create_shape
2018-11-14 17:30:18.589608 - ws-mode_change
2018-11-14 17:30:20.054064 - ws-scale_shape
2018-11-14 17:30:21.368781 - ws-scale_shape
2018-11-14 17:3

2018-11-14 17:05:51.392848 - ws-rotate_shape
2018-11-14 17:05:51.539588 - ws-rotate_shape
2018-11-14 17:05:51.684246 - ws-rotate_shape
2018-11-14 17:05:51.974593 - ws-rotate_shape
2018-11-14 17:05:52.119522 - ws-rotate_shape
2018-11-14 17:05:52.265358 - ws-rotate_shape
2018-11-14 17:05:52.410428 - ws-rotate_shape
2018-11-14 17:05:52.562135 - ws-rotate_shape
2018-11-14 17:05:52.848879 - ws-rotate_shape
2018-11-14 17:05:53.131531 - ws-rotate_shape
2018-11-14 17:05:53.275133 - ws-rotate_shape
2018-11-14 17:05:53.418170 - ws-rotate_shape
2018-11-14 17:05:53.709591 - ws-rotate_shape
2018-11-14 17:05:53.853314 - ws-rotate_shape
2018-11-14 17:05:53.998947 - ws-rotate_shape
2018-11-14 17:05:54.142952 - ws-rotate_shape
2018-11-14 17:05:54.286836 - ws-rotate_shape
2018-11-14 17:05:54.581915 - ws-rotate_shape
2018-11-14 17:05:54.869419 - ws-rotate_shape
2018-11-14 17:05:55.016656 - ws-rotate_shape
2018-11-14 17:05:55.164096 - ws-rotate_shape
2018-11-14 17:05:55.308281 - ws-rotate_shape
2018-11-14

2018-11-14 17:00:31.189230 - ws-players
2018-11-14 17:00:33.553540 - ws-create_user
2018-11-14 17:01:31.264195 - ws-puzzle_started
2018-11-14 17:01:32.797301 - ws-start_level
\start level\
{'task_id': 'Welcome!', 'set_id': 'Tutorial', 'fullscreen': True, 'resolution': {'x': 1366, 'y': 768}, 'conditions': '{"shapeLimits":[-1,1,0,0,0,0,0],"allowScale":true,"allowRotate":true,"gridDim":3}'}
2018-11-14 17:01:32.952044 - ws-click_nothing
2018-11-14 17:01:56.519148 - ws-mode_change
2018-11-14 17:01:59.178151 - ws-mode_change
2018-11-14 17:02:00.751216 - ws-create_shape
2018-11-14 17:02:40.685167 - ws-rotate_view
2018-11-14 17:02:41.705712 - ws-rotate_view
2018-11-14 17:02:47.481441 - ws-snapshot
2018-11-14 17:02:52.524179 - ws-rotate_view
2018-11-14 17:02:54.305171 - ws-rotate_view
2018-11-14 17:03:03.714893 - ws-rotate_view
2018-11-14 17:03:06.085478 - ws-rotate_view
2018-11-14 17:03:11.368661 - ws-rotate_view
2018-11-14 17:03:12.670661 - ws-rotate_view
2018-11-14 17:03:14.210043 - ws-rotat

2018-11-14 17:31:26.997916 - ws-move_shape
2018-11-14 17:31:28.711429 - ws-move_shape
2018-11-14 17:31:29.858394 - ws-move_shape
2018-11-14 17:31:33.027375 - ws-move_shape
2018-11-14 17:31:34.200600 - ws-move_shape
2018-11-14 17:31:34.734562 - ws-deselect_shape
2018-11-14 17:31:35.459670 - ws-select_shape
2018-11-14 17:31:37.190202 - ws-move_shape
2018-11-14 17:31:37.652948 - ws-deselect_shape
2018-11-14 17:31:38.830013 - ws-click_nothing
2018-11-14 17:31:41.658046 - ws-snapshot
2018-11-14 17:31:47.646167 - ws-rotate_view
2018-11-14 17:31:48.940909 - ws-rotate_view
2018-11-14 17:31:50.403760 - ws-select_shape
2018-11-14 17:31:53.720642 - ws-mode_change
2018-11-14 17:31:54.868991 - ws-delete_shape
2018-11-14 17:31:56.501919 - ws-rotate_view
2018-11-14 17:31:59.325705 - ws-mode_change
2018-11-14 17:32:03.435645 - ws-create_shape
2018-11-14 17:32:05.382398 - ws-move_shape
2018-11-14 17:32:06.394896 - ws-move_shape
2018-11-14 17:32:07.738074 - ws-move_shape
2018-11-14 17:32:08.670449 - ws-

2018-11-14 17:02:28.022123 - ws-deselect_shape
2018-11-14 17:02:37.367143 - ws-select_shape
2018-11-14 17:02:38.324377 - ws-move_shape
2018-11-14 17:02:39.142978 - ws-deselect_shape
2018-11-14 17:02:41.218098 - ws-select_shape
2018-11-14 17:02:42.174210 - ws-move_shape
2018-11-14 17:02:44.341003 - ws-move_shape
2018-11-14 17:03:01.302636 - ws-check_solution
2018-11-14 17:03:15.567561 - ws-move_shape
2018-11-14 17:03:27.506215 - ws-deselect_shape
2018-11-14 17:03:34.959484 - ws-check_solution
2018-11-14 17:03:39.874349 - ws-select_shape
2018-11-14 17:03:41.154468 - ws-move_shape
2018-11-14 17:03:45.795650 - ws-rotate_view
2018-11-14 17:03:47.406442 - ws-rotate_view
2018-11-14 17:03:49.741201 - ws-rotate_view
2018-11-14 17:03:51.685049 - ws-rotate_view
2018-11-14 17:03:55.892555 - ws-rotate_view
2018-11-14 17:03:59.865562 - ws-check_solution
2018-11-14 17:04:28.520385 - ws-rotate_view
2018-11-14 17:04:33.864281 - ws-rotate_view
2018-11-14 17:04:47.888226 - ws-rotate_view
2018-11-14 17:05

TypeError: must be str, not NoneType

In [109]:
puzzleDf = pd.DataFrame.from_dict(puzzleDict, orient='index')

In [110]:
puzzleDf['session_id'] = [item[0] for item in puzzleDf.index.str.split('~')]
puzzleDf['puzzle_id'] = [item[1] for item in puzzleDf.index.str.split('~')]

In [111]:
puzzleDf

,active_time,n_actions,n_attempts,n_complete,session_id,puzzle_id
08lyf80qem23ddyruo38pm1h1wpp3q6l-Playtest5-First~Good Luck!,2.62,33,3,1,08lyf80qem23ddyruo38pm1h1wpp3q6l-Playtest5-First,Good Luck!
08lyf80qem23ddyruo38pm1h1wpp3q6l-Playtest5-First~Limited Objects,0.00,0,0,0,08lyf80qem23ddyruo38pm1h1wpp3q6l-Playtest5-First,Limited Objects
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Deleting Objects,0.89,11,1,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Deleting Objects
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Good Luck!,3.95,13,2,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Good Luck!
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Intro Puzzle,0.90,4,1,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Intro Puzzle
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Intro Puzzle 2,1.32,8,1,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Intro Puzzle 2
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Intro Puzzle 3,23.72,236,6,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Intro Puzzle 3
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Limited Objects,0.88,8,1,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Limited Objects
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Moving Objects,2.50,30,4,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Moving Objects
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First~Moving the Camera,1.99,12,1,1,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,Moving the Camera
